In [2]:
import pandas as pd
import datetime
import xalpha as xa
from datetime import datetime, date, timedelta

In [241]:
def fixed_investment_week(code, start, end, amount):
    fund = xa.fundinfo(code)
    price_df = fund.price
    print(fund.info())
    
    start_index = price_df.loc[price_df['date'] == start].index[0]
    end_index = price_df.loc[price_df['date'] == end].index[0]
    buy_df = price_df.iloc[start_index : end_index] # 待买df
    sell_price = float(price_df.loc[price_df['date'] == end]['totvalue']) # 要卖那天的累计净值
    
    total = [0] * 5  # 存放对应星期合计买的份数
    count = [0] * 5  # 存放对应星期的成本

    for index, row in buy_df.iterrows():
        weekday = int(row['date'].weekday())
        # amount金额除以当日累计净值得到购买基金份额，并计算累计份额
        total[weekday] = total[weekday] + amount/float(row['totvalue']) 
        count[weekday] += amount

    for index, unit in enumerate(total):
        print(f"每周 {index+1} 定投，累计投入 {count[index]} 单位金额，",
              f"最终卖出 {round(sell_price*unit,2)} 单位金额，",
              f"收益率 {round(100*(sell_price*unit-count[index])/count[index],2)}% ;")



In [242]:
fixed_investment('005827', '2020-08-04', '2021-08-03', 100)

fund name: 易方达蓝筹精选混合
fund code: 005827
fund purchase fee: 0.15%
fund redemption fee info: ['小于7天', '1.50%', '大于等于7天，小于等于29天', '0.75%', '大于等于30天，小于等于364天', '0.50%', '大于等于365天，小于等于729天', '0.25%', '大于等于730天', '0.00%']
None
每周 1 定投，累计投入 4700 单位金额， 最终卖出 4483.31 单位金额， 收益率 -4.61% ;
每周 2 定投，累计投入 4900 单位金额， 最终卖出 4662.76 单位金额， 收益率 -4.84% ;
每周 3 定投，累计投入 4900 单位金额， 最终卖出 4663.98 单位金额， 收益率 -4.82% ;
每周 4 定投，累计投入 4900 单位金额， 最终卖出 4628.33 单位金额， 收益率 -5.54% ;
每周 5 定投，累计投入 4900 单位金额， 最终卖出 4655.94 单位金额， 收益率 -4.98% ;


In [243]:
def fixed_investment_month(code, start, end, amount, day_list=['05', '10', '15', '20', '25']):
    fund = xa.fundinfo(code)
    price_df = fund.price
    print(fund.info())
    
    start_index = price_df.loc[price_df['date'] == start].index[0]
    end_index = price_df.loc[price_df['date'] == end].index[0]
    buy_df = price_df.iloc[start_index : end_index] # 待买df
    sell_price = float(price_df.loc[price_df['date'] == end]['totvalue']) # 要卖那天的累计净值
    
    
    buy_df['year-month'] = buy_df['date'].map(lambda x: x.strftime("%Y-%m"))
    month_array = buy_df['year-month'].unique()

    shares_dict = {}  # 存放对应星期合计买的份数
    cost_dict = {}  # 存放对应星期的成本

    for month in month_array:
        for day in day_list:
            buy_day = datetime.strptime(f"{month}-{day} 0:0:0", "%Y-%m-%d %H:%M:%S")
            while buy_day not in buy_df['date'].to_list():
                # 如果那一天不交易就取前一个交易日
                buy_day = buy_day + timedelta(days = -1)

            shares_dict[day] = shares_dict.get(day, 0) + amount/float(buy_df.loc[buy_df['date'] == buy_day]['totvalue'])
            cost_dict[day] = cost_dict.get(day, 0) + amount
    
    for day in day_list:
        print(f"每月 {day} 日定投，累计投入 {cost_dict[day]} 单位金额，",
              f"最终卖出 {round(sell_price*shares_dict[day], 2)} 单位金额，",
              f"收益率 {round(100*(sell_price*shares_dict[day] - cost_dict[day])/cost_dict[day],2)}% ;")

    
    


In [244]:
fixed_investment_month('005827', '2020-08-04', '2021-08-03', 400)

fund name: 易方达蓝筹精选混合
fund code: 005827
fund purchase fee: 0.15%
fund redemption fee info: ['小于7天', '1.50%', '大于等于7天，小于等于29天', '0.75%', '大于等于30天，小于等于364天', '0.50%', '大于等于365天，小于等于729天', '0.25%', '大于等于730天', '0.00%']
None
每月 05 日定投，累计投入 5200 单位金额， 最终卖出 4926.81 单位金额， 收益率 -5.25% ;
每月 10 日定投，累计投入 5200 单位金额， 最终卖出 5007.46 单位金额， 收益率 -3.7% ;
每月 15 日定投，累计投入 5200 单位金额， 最终卖出 4979.01 单位金额， 收益率 -4.25% ;
每月 20 日定投，累计投入 5200 单位金额， 最终卖出 4932.95 单位金额， 收益率 -5.14% ;
每月 25 日定投，累计投入 5200 单位金额， 最终卖出 4925.19 单位金额， 收益率 -5.28% ;


In [3]:
fund = xa.fundinfo('005827')
price_df = fund.price
price_df.head()

,date,netvalue,comment,totvalue
0,2018-09-05,1.0000,0,1.0000
1,2018-09-07,0.9986,0,0.9986
2,2018-09-14,0.9985,0,0.9985
3,2018-09-21,1.0143,0,1.0143
4,2018-09-28,1.0192,0,1.0192


In [23]:
# a = fund.info()
dir(fund)
print('\n'.join(['%s:%s' % item for item in fund.__dict__.items()]))
# fund._url

_url:http://fund.eastmoney.com/pingzhongdata/005827.js
_feeurl:http://fund.eastmoney.com/f10/jjfl_005827.html
priceonly:False
code:005827
round_label:0
dividend_label:0
value_label:0
specialdate:[]
fenhongdate:[]
zhesuandate:[]
format:csv
_page:<Response [200]>
rate:0.15
name:易方达蓝筹精选混合
price:          date  netvalue  comment  totvalue
0   2018-09-05    1.0000        0    1.0000
1   2018-09-07    0.9986        0    0.9986
2   2018-09-14    0.9985        0    0.9985
3   2018-09-21    1.0143        0    1.0143
4   2018-09-28    1.0192        0    1.0192
5   2018-10-08    0.9972        0    0.9972
6   2018-10-09    0.9970        0    0.9970
7   2018-10-10    0.9885        0    0.9885
8   2018-10-11    0.9731        0    0.9731
9   2018-10-12    0.9872        0    0.9872
10  2018-10-15    0.9820        0    0.9820
11  2018-10-16    0.9793        0    0.9793
12  2018-10-18    0.9706        0    0.9706
13  2018-10-19    0.9855        0    0.9855
14  2018-10-22    1.0084        0    1.0084
15 

In [15]:
import sqlalchemy
dir(sqlalchemy)

['__add__',
 '__and__',
 '__bool__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__le__',
 '__lshift__',
 '__lt__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__radd__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmod__',
 '__rmul__',
 '__rshift__',
 '__rsub__',
 '__rtruediv__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__visit_name__',
 '__weakref__',
 '_allow_label_resolve',
 '_alt_names',
 '_annotate',
 '_annotations',
 '_bind_param',
 '_clone',
 '_cloned_set',
 '_compare_name_for_result',
 '_compiler',
 '_compiler_dispatch',
 '_constructor',
 '_copy_internals',
 '_deannotate',
 '_execute_on_connection',
 '_e

In [16]:
help(sqlalchemy.Column())

Help on Column in module sqlalchemy.sql.schema object:

class Column(sqlalchemy.sql.base.DialectKWArgs, SchemaItem, sqlalchemy.sql.elements.ColumnClause)
 |  Column(*args, **kwargs)
 |  
 |  Represents a column in a database table.
 |  
 |  Method resolution order:
 |      Column
 |      sqlalchemy.sql.base.DialectKWArgs
 |      SchemaItem
 |      sqlalchemy.sql.base.SchemaEventTarget
 |      sqlalchemy.sql.elements.ColumnClause
 |      sqlalchemy.sql.base.Immutable
 |      sqlalchemy.sql.elements.ColumnElement
 |      sqlalchemy.sql.operators.ColumnOperators
 |      sqlalchemy.sql.operators.Operators
 |      sqlalchemy.sql.elements.ClauseElement
 |      sqlalchemy.sql.visitors.Visitable
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, *args, **kwargs)
 |      Construct a new ``Column`` object.
 |      
 |      :param name: The name of this column as represented in the database.
 |        This argument may be the first positional argument, or specified
 | 